In [1]:
%matplotlib inline
""" handling files support packages """
from glob import glob

""" logic support packages """
import numpy as np
import pandas as pd

""" plot support packages """
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython.display import Image
import pydotplus

""" image trasformation packages """
from PIL import Image
import skimage.io as skio

""" statistical data visualization packages"""
import seaborn as sns

""" machine learning functions """
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.externals import joblib

""" seaborn configurations """
sns.set_style('white')
sns.set_context('talk')
plt.rcParams['figure.figsize'] = 20, 10

# Tentativa de com base equilibrada 30%

In [2]:
combinacoes30 = pd.read_csv("../combinacoes30.txt", header = 0)

In [3]:
le = preprocessing.LabelEncoder()
nomes = []
for i in sorted(glob('../letras/caracter*.png')):
    nomes.append(i.split("/")[1] + "/" + i.split("/")[2])

In [4]:
le.fit(nomes)

LabelEncoder()

In [5]:
combinacoes30["char1"] = le.transform(combinacoes30["char1"])
combinacoes30["char2"] = le.transform(combinacoes30["char2"])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(combinacoes30[["char1", "char2", "MSE", "ISS", "MSE_centro", "ISS_centro"]],
                                                                    combinacoes30["resposta"], test_size=0.3, random_state=0)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(combinacoes30[["MSE", "ISS", "MSE_centro", "ISS_centro"]],
                                                                    combinacoes30["resposta"], test_size=0.3, random_state=0)

## SVM

In [8]:
clf = SVC()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.69871650529160101

## Decision Tree

In [9]:
from sklearn import tree

clf1 = tree.DecisionTreeClassifier()

clf1.fit( X = X_train, y = y_train )
print( clf1.score(X_test, y_test) )

0.634316595361


In [10]:
# Tentativa de plotar a árvore do modelo acima
#dot_data = tree.export_graphviz(clf1, out_file=None, 
#                         feature_names = ["MSE","ISS",
#                                          "MSE_centro","ISS_centro"],  
#                         class_names = combinacoes30["resposta"],  
#                         filled = True, rounded = True,  
#                         special_characters=True)

#graph = pydotplus.graph_from_dot_data(dot_data)  
#Image(graph.create_png())
##graph.write_pdf("perfil_anovalor.pdf")

# Tentativa com base inteira

In [11]:
combinacoes = pd.read_csv("../combinacoes.txt", header = 0)

In [12]:
combinacoes["char1"] = le.transform(combinacoes["char1"])
combinacoes["char2"] = le.transform(combinacoes["char2"])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(combinacoes[["char1", "char2", "MSE", "ISS", "MSE_centro", "ISS_centro"]],
                                                                    combinacoes["resposta"], test_size=0.3, random_state=0)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(combinacoes[["MSE", "ISS", "MSE_centro", "ISS_centro"]],
                                                                    combinacoes["resposta"], test_size=0.3, random_state=0)

## SVM

In [15]:
#clf.fit(X_train, y_train)
#clf.score(X_test, y_test)

# Decision Tree

In [16]:
%%time
clf1.fit( X = X_train, y = y_train )
print( clf1.score(X_test, y_test) )

0.964306532731
CPU times: user 1.9 s, sys: 0 ns, total: 1.9 s
Wall time: 1.9 s


In [17]:
# Tentativa de plotar a árvore do modelo acima
#dot_data = tree.export_graphviz(clf1, out_file=None, 
#                         feature_names = ["char1", "char2","MSE","ISS","MSE_centro","ISS_centro"],  
#                         class_names = combinacoes["resposta"],  
#                         filled = True, rounded = True,  
#                         special_characters=True)

#graph = pydotplus.graph_from_dot_data(dot_data)  
#Image(graph.create_png())
##graph.write_pdf("perfil_anovalor.pdf")

## Random Forest

In [18]:
from sklearn.ensemble import RandomForestClassifier
clfRF = RandomForestClassifier()

In [19]:
%%time
clfRF.fit( X = X_train, y = y_train )
print( clfRF.score(X_test, y_test) )

0.982417612211
CPU times: user 6.54 s, sys: 4 ms, total: 6.55 s
Wall time: 6.55 s


## Binarizando o melhor modelo

In [20]:
joblib.dump(clf1, 'classifier.pkl') 

['classifier.pkl']

In [21]:
joblib.dump(clfRF, 'classifierRF.pkl') 

['classifierRF.pkl']

## testes temporários

In [22]:
# alguns testes temporários
from funcoes import ler_letras, compare_images
_, letters_dict = ler_letras("../letras.csv")

img1 = skio.imread("../" + 'letras/caracter0.png')
img2 = skio.imread("../" + 'letras/caracter1.png')

m, s = compare_images(img1, img2)
mc, cs = compare_images(img1[10:40,], img2[10:40,])
    
print(clf1.predict([[m, s, mc, cs]])[0] == 1)

TabError: inconsistent use of tabs and spaces in indentation (funcoes.py, line 307)